In [1]:
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
from datetime import datetime

import pandas as pd # type: ignore

import src.fetch_data # type: ignore
import src.core_transform # type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore
import src.output_to_excel
import cdutils.loans.calculations # type: ignore
# import cdutils.selo # type: ignore
from src._version import __version__


data = src.fetch_data.fetch_data()

In [2]:
acctcommon = data['wh_acctcommon'].copy()
org = data['wh_org'].copy()

In [3]:
org

,orgnbr,orgname,orgtypcd,orgtypcddesc,taxid,taxidtypcd,rpt1099intyn,privacyyn,taxexemptyn,cipratingcd,...,sicsubcd,sicsubcddesc,naicscd,naicscddesc,adddate,datelastmaint,rundate,allowpromoyn,homeemail,busemail
0,1005119,ALINE M MULHERN FAMILY RVOC TR,TRST,Trust/Fiduciary,A2146D547180467A51E37F894252AE9A,FEIN,Y,N,N,None,...,None,None,525920,"Trusts, Estates, and Agency Accounts",2017-02-24,2025-07-02 21:22:00,2025-07-02,N,martyfontes@windstream.net,None
1,1005120,FLINTSTONE REALTY LLC,LLC,Limited Liability Corporation,BD09EC1260FBDB68F0110FAA2848B1BC,FEIN,Y,N,N,None,...,None,None,531390,Other Activities Related to Real Estate,2017-12-13,2025-07-02 21:22:00,2025-07-02,N,bedrockcarol@gmail.com,None
2,1005121,JOHN EDWARD GALVIN JR ESTATE,EST,Estate,B670ACB1B880CF95033E6A95F37DAC84,FEIN,Y,N,N,None,...,None,None,525920,"Trusts, Estates, and Agency Accounts",2016-03-23,2025-07-02 21:22:00,2025-07-02,N,None,None
3,1005122,RICHARD A MASTRIA JR IRR SPOUSAL ACC TR,TRST,Trust/Fiduciary,F10C7B12D7F5979D1DFB67FE50D303FE,FEIN,Y,N,N,None,...,None,None,531390,Other Activities Related to Real Estate,2017-12-20,2025-07-02 21:22:00,2025-07-02,N,None,ede@mastria.com
4,1005123,WASHINGTON STREET REALTY LLC,LLC,Limited Liability Corporation,C2206894DF0D9356EE11204805DF41B6,FEIN,Y,N,N,None,...,None,None,531110,Lessors of Residential Buildings and Dwellings,2017-01-14,2025-07-02 21:22:00,2025-07-02,N,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17066,1011284,DENISE BOUCHER MARITAL TRUST,EST,Estate,601681025EC9A2102786B199B2EEE150,FEIN,Y,N,N,None,...,None,None,525920,"Trusts, Estates, and Agency Accounts",2013-07-11,2025-07-02 21:22:01,2025-07-02,N,None,None
17067,1011285,DENISE BOUCHER FAMILY TRUST,EST,Estate,2390B1205A93ACA71483976468BFDC06,FEIN,Y,N,N,None,...,None,None,525920,"Trusts, Estates, and Agency Accounts",2013-07-11,2025-07-02 21:22:01,2025-07-02,N,None,duane@boucherrealestate.com
17068,1011286,DTD 01/07/2005,CORP,Corporation/Business,601681025EC9A2102786B199B2EEE150,FEIN,Y,N,N,None,...,None,None,None,None,2013-07-11,2025-07-02 21:22:01,2025-07-02,N,None,None
17069,1011287,SHEARS OF CUMBERLAND INC,CORP,Corporation/Business,CCBC7D3E783F846A5CE201FC6533DF7D,FEIN,Y,N,N,None,...,None,None,812112,Beauty Salons,2020-09-25,2025-07-02 21:22:01,2025-07-02,N,None,shearsofcumberland@icloud.com


In [43]:
org = org.sort_values(by='adddate', ascending=False).copy()

In [44]:
import cdutils.deduplication

dedupe_list = [
    {'df':org, 'field':'orgnbr'}
]

org = cdutils.deduplication.dedupe(dedupe_list)

In [45]:
assert org['orgnbr'].is_unique, "Fail"


In [46]:
org['orgtypcddesc'].unique()

array(['Trust/Fiduciary', 'Limited Liability Corporation', 'Estate',
       'Sole Proprietorship', 'Other', 'Partnership',
       'Corporation/Business', 'Association / Society',
       'Insurance Receiver', 'Company', 'Real Estate Investment Trust',
       'Investor', 'Dealer', 'Bank Branch', 'Law Firm', None, 'Business',
       'Doing Business As', 'Private Corporation',
       'Non-Profit/Charitable', 'Real Estate Agent', 'Tax Receiver',
       'Municipal', 'Religious', 'Tax Servicer', 'Limited Partnership',
       'Credit Bureau', 'Governmental', 'Educational', 'Prospect',
       'Limited Liability Partnership', 'Insurance Agent', 'Vendor Forms',
       'Legal', 'Federal Home Loan Bank', 'School District', 'Region',
       'Co-operative Bank', 'Commercial/Other Bank', 'Flood Vendor',
       'Credit Reporting Agency', 'Credit Union', 'Do Not use Org 1'],
      dtype=object)

In [47]:
valid_org_list = [
    'Limited Liability Corporation',
    'Corporation/Business',
    'Real Estate Investment Trust',
    'Law Firm',
    'Doing Business As',
    'Business',
    'Private Corporation',
    'Company',
    'Partnership',
    'Limited Partnership',
    'Limited Liability Partnership'
]
org_filtered = org[org['orgtypcddesc'].isin(valid_org_list)]

In [48]:
org_filtered['orgnbr'].nunique()

10824

In [49]:
acctcommon_deposits = acctcommon[acctcommon['loanofficer'].isnull()].copy()

In [50]:
merged_df = pd.merge(org_filtered, acctcommon_deposits, left_on='orgnbr', right_on='taxrptfororgnbr', how='inner')

In [51]:
merged_df

,orgnbr,orgname,orgtypcd,orgtypcddesc,taxid,taxidtypcd,rpt1099intyn,privacyyn,taxexemptyn,cipratingcd,...,curracctstatcd,noteintrate,bookbalance,notebal,contractdate,datemat,taxrptfororgnbr,taxrptforpersnbr,loanofficer,acctofficer
0,1018465,CREATEYOROWN STUDIO LLC,LLC,Limited Liability Corporation,70F1D4F8943D2197F92895439683F03D,FEIN,Y,N,N,None,...,ACT,0,200,200,2025-06-30,NaT,1018465.0,NaN,None,JACQUELINE A. THEIS
1,1018461,DIGHTON BOATWORKS LLC,PART,Partnership,491AAB13C420802973FD0D0328C741CF,FEIN,Y,N,N,None,...,ACT,0.0003,4000.01,4000.01,2025-06-27,NaT,1018461.0,NaN,None,JOHN G. DUGGAN
2,1018459,CHAVES HOLDINGS INC,CORP,Corporation/Business,0F2D38EA8EDAFEEE78EB3CE5ABCC5EAD,FEIN,Y,N,N,None,...,ACT,0.0305,32166763.22,32166763.22,2025-07-02,NaT,1018459.0,NaN,None,None
3,1018459,CHAVES HOLDINGS INC,CORP,Corporation/Business,0F2D38EA8EDAFEEE78EB3CE5ABCC5EAD,FEIN,Y,N,N,None,...,ACT,0.0095,10,10,2025-06-27,NaT,1018459.0,NaN,None,RICHARD J. CLARK
4,1018445,TAKEOFF PLANO LLC,LLC,Limited Liability Corporation,00102C32CD900CB5A3BDC7C2C3B292B5,FEIN,Y,N,N,None,...,ACT,0,200,200,2025-06-23,NaT,1018445.0,NaN,None,CHRISTINE M. PAREDES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,1002626,ROTARY CLUB OF TAUNTON INC,CORP,Corporation/Business,736A97090B7851E7EAD1855C71F53409,FEIN,N,N,N,None,...,DORM,0.0005,12118.63,12118.63,2020-09-24,NaT,1002626.0,NaN,None,INTERNAL ACCOUNTS DIRECTORS
6011,1002626,ROTARY CLUB OF TAUNTON INC,CORP,Corporation/Business,736A97090B7851E7EAD1855C71F53409,FEIN,N,N,N,None,...,ACT,0.0365,11882.46,11882.46,2001-05-29,2026-05-27,1002626.0,NaN,None,INTERNAL ACCOUNTS DIRECTORS
6012,1002626,ROTARY CLUB OF TAUNTON INC,CORP,Corporation/Business,736A97090B7851E7EAD1855C71F53409,FEIN,N,N,N,None,...,ACT,0.0365,10964.84,10964.84,2023-06-21,2026-06-20,1002626.0,NaN,None,INTERNAL ACCOUNTS DIRECTORS
6013,1002626,ROTARY CLUB OF TAUNTON INC,CORP,Corporation/Business,736A97090B7851E7EAD1855C71F53409,FEIN,N,N,N,None,...,ACT,0.0026,1978,1978,1987-05-13,NaT,1002626.0,NaN,None,INTERNAL ACCOUNTS DIRECTORS


In [52]:
dedupe_list = [
    {'df':merged_df, 'field':'orgnbr'}
]

merged_df = cdutils.deduplication.dedupe(dedupe_list).copy()

In [53]:
merged_df

,orgnbr,orgname,orgtypcd,orgtypcddesc,taxid,taxidtypcd,rpt1099intyn,privacyyn,taxexemptyn,cipratingcd,...,curracctstatcd,noteintrate,bookbalance,notebal,contractdate,datemat,taxrptfororgnbr,taxrptforpersnbr,loanofficer,acctofficer
0,1018465,CREATEYOROWN STUDIO LLC,LLC,Limited Liability Corporation,70F1D4F8943D2197F92895439683F03D,FEIN,Y,N,N,None,...,ACT,0,200,200,2025-06-30,NaT,1018465.0,NaN,None,JACQUELINE A. THEIS
1,1018461,DIGHTON BOATWORKS LLC,PART,Partnership,491AAB13C420802973FD0D0328C741CF,FEIN,Y,N,N,None,...,ACT,0.0003,4000.01,4000.01,2025-06-27,NaT,1018461.0,NaN,None,JOHN G. DUGGAN
2,1018459,CHAVES HOLDINGS INC,CORP,Corporation/Business,0F2D38EA8EDAFEEE78EB3CE5ABCC5EAD,FEIN,Y,N,N,None,...,ACT,0.0305,32166763.22,32166763.22,2025-07-02,NaT,1018459.0,NaN,None,None
4,1018445,TAKEOFF PLANO LLC,LLC,Limited Liability Corporation,00102C32CD900CB5A3BDC7C2C3B292B5,FEIN,Y,N,N,None,...,ACT,0,200,200,2025-06-23,NaT,1018445.0,NaN,None,CHRISTINE M. PAREDES
5,1018437,100 WESTMINSTER PARTNERS LLC,LLC,Limited Liability Corporation,E8C762FB9CB67E5E5FC55936996027C0,FEIN,Y,N,N,None,...,ACT,0.0245,503396.31,503396.31,2025-06-20,NaT,1018437.0,NaN,None,MOBOLAJI OMISORE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,1002690,REHOBOTH LIONS CLUB INC,CORP,Corporation/Business,668993D69EF833B492CFB12E1E12DF40,FEIN,N,N,N,None,...,ACT,0.045,20741.28,20741.28,1994-12-19,2025-12-16,1002690.0,NaN,None,FRANK P. WILHELM
5998,1002714,RAYNHAM LIONS CLUB INC,CORP,Corporation/Business,78C39F40A6D4133AB6374A799CAD9E6B,FEIN,N,N,N,None,...,ACT,0.0036,3433.44,3433.44,1996-02-01,NaT,1002714.0,NaN,None,AMY M. BRIGGS
6002,1003188,REALTOR ASSOCIATION OF SOUTHEASTERN MASSACHUSE...,CORP,Corporation/Business,FC2BE638C19E880470E62DC622134D6B,FEIN,Y,N,N,None,...,ACT,0.0005,1883.67,1883.67,2016-06-06,NaT,1003188.0,NaN,None,JUSTIN A. JEFFREY
6008,1002918,POND VIEW EXCAVATION CORP,CORP,Corporation/Business,4BC81EAADC6E97F7AD443F6C411C5B64,FEIN,N,N,N,None,...,ACT,0,57033.74,57033.74,2021-09-13,NaT,1002918.0,NaN,None,FRANK P. WILHELM


In [54]:
# Cleaning to get first 2 digits
merged_df['naics_sector'] = merged_df['naicscd'].astype(str).str[:2]

In [55]:
sector_map = {
    "11": "Agriculture, Forestry, Fishing and Hunting",
    "21": "Mining, Quarrying, and Oil and Gas Extraction",
    "22": "Utilities",
    "23": "Construction",
    "31": "Manufacturing",
    "32": "Manufacturing",
    "33": "Manufacturing",
    "42": "Wholesale Trade",
    "45": "Retail Trade",
    "45": "Retail Trade",
    "48": "Transportation and Warehousing",
    "49": "Transportation and Warehousing",
    "51": "Information",
    "52": "Finance and Insurance",
    "53": "Real Estate and Rental and Leasing",
    "54": "Professional, Scientific, and Technical Services",
    "55": "Management of Companies and Enterprises",
    "56": "Administrative and Support and Waste Management and Remediation Services",
    "61": "Educational Services",
    "62": "Health Care and Social Assistance",
    "71": "Arts, Entertainment, and Recreation",
    "72": "Accommodation and Food Services",
    "81": "Other",
    "92": "Public Administration"
}

In [56]:
merged_df['sector_desc'] = merged_df['naics_sector'].map(sector_map).fillna('Other')

In [57]:
merged_df

,orgnbr,orgname,orgtypcd,orgtypcddesc,taxid,taxidtypcd,rpt1099intyn,privacyyn,taxexemptyn,cipratingcd,...,bookbalance,notebal,contractdate,datemat,taxrptfororgnbr,taxrptforpersnbr,loanofficer,acctofficer,naics_sector,sector_desc
0,1018465,CREATEYOROWN STUDIO LLC,LLC,Limited Liability Corporation,70F1D4F8943D2197F92895439683F03D,FEIN,Y,N,N,None,...,200,200,2025-06-30,NaT,1018465.0,NaN,None,JACQUELINE A. THEIS,81,Other
1,1018461,DIGHTON BOATWORKS LLC,PART,Partnership,491AAB13C420802973FD0D0328C741CF,FEIN,Y,N,N,None,...,4000.01,4000.01,2025-06-27,NaT,1018461.0,NaN,None,JOHN G. DUGGAN,No,Other
2,1018459,CHAVES HOLDINGS INC,CORP,Corporation/Business,0F2D38EA8EDAFEEE78EB3CE5ABCC5EAD,FEIN,Y,N,N,None,...,32166763.22,32166763.22,2025-07-02,NaT,1018459.0,NaN,None,None,54,"Professional, Scientific, and Technical Services"
4,1018445,TAKEOFF PLANO LLC,LLC,Limited Liability Corporation,00102C32CD900CB5A3BDC7C2C3B292B5,FEIN,Y,N,N,None,...,200,200,2025-06-23,NaT,1018445.0,NaN,None,CHRISTINE M. PAREDES,71,"Arts, Entertainment, and Recreation"
5,1018437,100 WESTMINSTER PARTNERS LLC,LLC,Limited Liability Corporation,E8C762FB9CB67E5E5FC55936996027C0,FEIN,Y,N,N,None,...,503396.31,503396.31,2025-06-20,NaT,1018437.0,NaN,None,MOBOLAJI OMISORE,53,Real Estate and Rental and Leasing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,1002690,REHOBOTH LIONS CLUB INC,CORP,Corporation/Business,668993D69EF833B492CFB12E1E12DF40,FEIN,N,N,N,None,...,20741.28,20741.28,1994-12-19,2025-12-16,1002690.0,NaN,None,FRANK P. WILHELM,81,Other
5998,1002714,RAYNHAM LIONS CLUB INC,CORP,Corporation/Business,78C39F40A6D4133AB6374A799CAD9E6B,FEIN,N,N,N,None,...,3433.44,3433.44,1996-02-01,NaT,1002714.0,NaN,None,AMY M. BRIGGS,81,Other
6002,1003188,REALTOR ASSOCIATION OF SOUTHEASTERN MASSACHUSE...,CORP,Corporation/Business,FC2BE638C19E880470E62DC622134D6B,FEIN,Y,N,N,None,...,1883.67,1883.67,2016-06-06,NaT,1003188.0,NaN,None,JUSTIN A. JEFFREY,81,Other
6008,1002918,POND VIEW EXCAVATION CORP,CORP,Corporation/Business,4BC81EAADC6E97F7AD443F6C411C5B64,FEIN,N,N,N,None,...,57033.74,57033.74,2021-09-13,NaT,1002918.0,NaN,None,FRANK P. WILHELM,23,Construction


In [58]:
from datetime import datetime

merged_df['adddate'] = pd.to_datetime(merged_df['adddate'], errors='coerce')

In [59]:
merged_df['tenure'] = (pd.to_datetime('today') - merged_df['adddate'])

In [60]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3863 entries, 0 to 6009
Data columns (total 44 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   orgnbr            3863 non-null   int64          
 1   orgname           3863 non-null   object         
 2   orgtypcd          3863 non-null   object         
 3   orgtypcddesc      3863 non-null   object         
 4   taxid             3860 non-null   object         
 5   taxidtypcd        3860 non-null   object         
 6   rpt1099intyn      3863 non-null   object         
 7   privacyyn         3863 non-null   object         
 8   taxexemptyn       3863 non-null   object         
 9   cipratingcd       24 non-null     object         
 10  creditscore       70 non-null     float64        
 11  siccd             165 non-null    object         
 12  siccddesc         165 non-null    object         
 13  sicsubcd          155 non-null    object         
 14  sicsubcddesc 

In [61]:
merged_df['tenure_years'] = merged_df['tenure'] / pd.Timedelta(days=365.25)

In [62]:
merged_df['tenure_years'].mean()

np.float64(10.197922762793167)

In [63]:
merged_df

,orgnbr,orgname,orgtypcd,orgtypcddesc,taxid,taxidtypcd,rpt1099intyn,privacyyn,taxexemptyn,cipratingcd,...,contractdate,datemat,taxrptfororgnbr,taxrptforpersnbr,loanofficer,acctofficer,naics_sector,sector_desc,tenure,tenure_years
0,1018465,CREATEYOROWN STUDIO LLC,LLC,Limited Liability Corporation,70F1D4F8943D2197F92895439683F03D,FEIN,Y,N,N,None,...,2025-06-30,NaT,1018465.0,NaN,None,JACQUELINE A. THEIS,81,Other,3 days 10:54:51.531335,0.009459
1,1018461,DIGHTON BOATWORKS LLC,PART,Partnership,491AAB13C420802973FD0D0328C741CF,FEIN,Y,N,N,None,...,2025-06-27,NaT,1018461.0,NaN,None,JOHN G. DUGGAN,No,Other,6 days 10:54:51.531335,0.017672
2,1018459,CHAVES HOLDINGS INC,CORP,Corporation/Business,0F2D38EA8EDAFEEE78EB3CE5ABCC5EAD,FEIN,Y,N,N,None,...,2025-07-02,NaT,1018459.0,NaN,None,None,54,"Professional, Scientific, and Technical Services",6 days 10:54:51.531335,0.017672
4,1018445,TAKEOFF PLANO LLC,LLC,Limited Liability Corporation,00102C32CD900CB5A3BDC7C2C3B292B5,FEIN,Y,N,N,None,...,2025-06-23,NaT,1018445.0,NaN,None,CHRISTINE M. PAREDES,71,"Arts, Entertainment, and Recreation",10 days 10:54:51.531335,0.028624
5,1018437,100 WESTMINSTER PARTNERS LLC,LLC,Limited Liability Corporation,E8C762FB9CB67E5E5FC55936996027C0,FEIN,Y,N,N,None,...,2025-06-20,NaT,1018437.0,NaN,None,MOBOLAJI OMISORE,53,Real Estate and Rental and Leasing,13 days 10:54:51.531335,0.036837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,1002690,REHOBOTH LIONS CLUB INC,CORP,Corporation/Business,668993D69EF833B492CFB12E1E12DF40,FEIN,N,N,N,None,...,1994-12-19,2025-12-16,1002690.0,NaN,None,FRANK P. WILHELM,81,Other,13052 days 10:54:51.531335,35.735674
5998,1002714,RAYNHAM LIONS CLUB INC,CORP,Corporation/Business,78C39F40A6D4133AB6374A799CAD9E6B,FEIN,N,N,N,None,...,1996-02-01,NaT,1002714.0,NaN,None,AMY M. BRIGGS,81,Other,13052 days 10:54:51.531335,35.735674
6002,1003188,REALTOR ASSOCIATION OF SOUTHEASTERN MASSACHUSE...,CORP,Corporation/Business,FC2BE638C19E880470E62DC622134D6B,FEIN,Y,N,N,None,...,2016-06-06,NaT,1003188.0,NaN,None,JUSTIN A. JEFFREY,81,Other,13052 days 10:54:51.531335,35.735674
6008,1002918,POND VIEW EXCAVATION CORP,CORP,Corporation/Business,4BC81EAADC6E97F7AD443F6C411C5B64,FEIN,N,N,N,None,...,2021-09-13,NaT,1002918.0,NaN,None,FRANK P. WILHELM,23,Construction,13052 days 10:54:51.531335,35.735674


In [64]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3863 entries, 0 to 6009
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   orgnbr            3863 non-null   int64          
 1   orgname           3863 non-null   object         
 2   orgtypcd          3863 non-null   object         
 3   orgtypcddesc      3863 non-null   object         
 4   taxid             3860 non-null   object         
 5   taxidtypcd        3860 non-null   object         
 6   rpt1099intyn      3863 non-null   object         
 7   privacyyn         3863 non-null   object         
 8   taxexemptyn       3863 non-null   object         
 9   cipratingcd       24 non-null     object         
 10  creditscore       70 non-null     float64        
 11  siccd             165 non-null    object         
 12  siccddesc         165 non-null    object         
 13  sicsubcd          155 non-null    object         
 14  sicsubcddesc 

In [65]:
merged_df = merged_df[[
    'orgnbr',
    'orgname',
    'orgtypcd',
    'orgtypcddesc',
    'taxid',
    'taxidtypcd',
    'naicscd',
    'naicscddesc',
    'adddate',
    'allowpromoyn',
    'homeemail',
    'busemail',
    'naics_sector',
    'sector_desc',
    'tenure',
    'tenure_years'
]].copy()

In [66]:
acctcommon_deposits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66205 entries, 1 to 89779
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   effdate           66205 non-null  datetime64[ns]
 1   acctnbr           66205 non-null  int64         
 2   ownersortname     66205 non-null  object        
 3   product           66205 non-null  object        
 4   noteopenamt       62787 non-null  object        
 5   ratetypcd         48171 non-null  object        
 6   mjaccttypcd       66205 non-null  object        
 7   currmiaccttypcd   66205 non-null  object        
 8   curracctstatcd    66205 non-null  object        
 9   noteintrate       64414 non-null  object        
 10  bookbalance       66205 non-null  object        
 11  notebal           66205 non-null  object        
 12  contractdate      66204 non-null  datetime64[ns]
 13  datemat           9491 non-null   datetime64[ns]
 14  taxrptfororgnbr   9179 non-

In [67]:
import pandas as pd
def add_net_balance_sum(df, account_df, org_key='orgnbr', balance_col='notebal'):
    # Group account_df by orgnbr and sum the 'Net Balance'
    balance_sum = account_df.groupby('taxrptfororgnbr')[balance_col].sum().reset_index()
    balance_sum.rename(columns={balance_col: 'Total Net Balance'}, inplace=True)
    # Left join original df with the aggregated sums
    result_df = pd.merge(df, balance_sum, left_on=org_key, right_on='taxrptfororgnbr', how='left')
    # If there were orgnbrs with no matching accounts, fill NaN with 0
    result_df['Total Net Balance'] = result_df['Total Net Balance'].fillna(0)
    return result_df

In [68]:
merged_df = add_net_balance_sum(merged_df, acctcommon_deposits)

In [88]:
merged_df.tail()

,orgnbr,orgname,orgtypcd,orgtypcddesc,taxid,taxidtypcd,naicscd,naicscddesc,adddate,allowpromoyn,homeemail,busemail,naics_sector,sector_desc,tenure,tenure_years,taxrptfororgnbr,Total Net Balance
3858,1002690,REHOBOTH LIONS CLUB INC,CORP,Corporation/Business,668993D69EF833B492CFB12E1E12DF40,FEIN,813410,Civic and Social Organizations,1989-10-08,N,None,None,81,Other,13052 days 10:54:51.531335,35.735674,1002690.0,174198.87
3859,1002714,RAYNHAM LIONS CLUB INC,CORP,Corporation/Business,78C39F40A6D4133AB6374A799CAD9E6B,FEIN,813410,Civic and Social Organizations,1989-10-08,N,raynhamlions@verizon.net,omihoff@verizon.net,81,Other,13052 days 10:54:51.531335,35.735674,1002714.0,19803.00
3860,1003188,REALTOR ASSOCIATION OF SOUTHEASTERN MASSACHUSE...,CORP,Corporation/Business,FC2BE638C19E880470E62DC622134D6B,FEIN,813990,"Other Similar Organizations (except Business, ...",1989-10-08,N,paul@rasem.realtor,None,81,Other,13052 days 10:54:51.531335,35.735674,1003188.0,257028.97
3861,1002918,POND VIEW EXCAVATION CORP,CORP,Corporation/Business,4BC81EAADC6E97F7AD443F6C411C5B64,FEIN,238190,"Other Foundation, Structure, and Building Exte...",1989-10-08,N,None,kim@foleyexcavation.com,23,Construction,13052 days 10:54:51.531335,35.735674,1002918.0,57033.74
3862,1002626,ROTARY CLUB OF TAUNTON INC,CORP,Corporation/Business,736A97090B7851E7EAD1855C71F53409,FEIN,813410,Civic and Social Organizations,1989-10-08,N,mcody@jcl-cpa.com,None,81,Other,13052 days 10:54:51.531335,35.735674,1002626.0,100608.95


In [70]:
merged_df = merged_df[merged_df['Total Net Balance']<1000000].copy()

In [71]:
merged_df['Total Net Balance'].mean()

np.float64(74140.30277299623)

In [72]:
grouping = merged_df.groupby('sector_desc')['orgnbr'].nunique().sort_values(ascending=False).reset_index()

In [73]:
grouping

,sector_desc,orgnbr
0,Real Estate and Rental and Leasing,986
1,Other,730
2,Construction,392
3,"Professional, Scientific, and Technical Services",244
4,Manufacturing,167
5,Health Care and Social Assistance,167
6,Accommodation and Food Services,165
7,Finance and Insurance,148
8,Administrative and Support and Waste Managemen...,132
9,"Arts, Entertainment, and Recreation",108


In [74]:
NAICS_BUSINESS_BREAKDOWN = Path('./output/naics_business_breakdown.csv')
grouping.to_csv(NAICS_BUSINESS_BREAKDOWN, index=False)

In [80]:
def summarize_by_sector(result_df):
    summary_df = result_df.groupby('sector_desc').agg(
        unique_orgs=('orgnbr','nunique'),
        total_deposit_balance=('Total Net Balance','mean'),
        avg_tenure_years=('tenure_years','mean')
    ).reset_index()

    return summary_df

In [81]:
final_summary = summarize_by_sector(merged_df)

In [85]:
FINAL_SUMMARY_PATH = Path('./output/full_summary.csv')
final_summary.to_csv(FINAL_SUMMARY_PATH, index=False)

In [86]:
final_summary

,sector_desc,unique_orgs,total_deposit_balance,avg_tenure_years
0,Accommodation and Food Services,165,67021.726848,8.087496
1,Administrative and Support and Waste Managemen...,132,72142.525076,10.661565
2,"Agriculture, Forestry, Fishing and Hunting",40,88722.4405,8.010417
3,"Arts, Entertainment, and Recreation",108,56956.531019,9.928743
4,Construction,392,87701.983418,9.380130
5,Educational Services,49,59648.397959,11.131503
6,Finance and Insurance,148,58819.268243,12.409277
7,Health Care and Social Assistance,167,79647.837425,9.942135
8,Information,22,85073.875909,7.748865
9,Management of Companies and Enterprises,43,68333.697674,7.083046


In [84]:
merged_df['Total Net Balance'].mean()

np.float64(74140.30277299623)

In [87]:
RAW_DATA_PATH = Path('./output/raw_data.csv')
merged_df.to_csv(RAW_DATA_PATH, index=False)